In [173]:
import pandas as pd
import os
from obspy import UTCDateTime
from obspy.geodetics.base import gps2dist_azimuth,kilometer2degrees
from obspy.clients.fdsn.client import Client
client = Client("IRIS")

In [196]:
name = ['Lon','Lat','Depth','Mag','strike1','dip1','rake1','strike2','dip2','rake2','mant','Exp','Date','Name']
meca = pd.read_csv('meca.gmt',sep='\s+',names=name,skiprows=33210)

In [197]:
SKKSEvent = pd.read_csv('EventList.csv')
Station = pd.read_csv('/Volumes/home/Research/GMT/Station_distribution/Station_info.csv')

In [198]:
def checktime(time):
    origin = time.rsplit(':')[0]
    newmm  = str((int(time.rsplit(':')[1])+1)).zfill(2)
    newss  = '00'
    ntime  = f'{origin}:{newmm}:{newss}'
    return ntime

In [203]:
for i in range(len(SKKSEvent)):
    STA = SKKSEvent['Station'][i]
    event   = SKKSEvent['Event'][i]
    yyyymmdd= event.rsplit('_')[0]
    hhmmss  = event.rsplit('_')[1]
    UTCtime = UTCDateTime(f'{yyyymmdd}{hhmmss}')
    
    stalon  = Station['lon'][Station['Station'] == STA].values[0]
    stalat  = Station['lat'][Station['Station'] == STA].values[0]
    for j in range(len(meca)):
        time = meca['Date'][j][:-2]
        if int(time[-2:]) ==60:
            time = checktime(time) 
        if UTCDateTime(time) == UTCtime:
            eventmeca = meca[j:j+1]
            evtlon = eventmeca['Lon'].values[0]
            evtlat = eventmeca['Lat'].values[0]
            evtdep = eventmeca['Depth'].values[0]
            strike = eventmeca['strike1'].values[0]
            dip    = eventmeca['dip1'].values[0]
            rake   = eventmeca['rake1'].values[0]
            dist,az,baz = gps2dist_azimuth(evtlat,evtlon,stalat,stalon)
            deg    = kilometer2degrees(dist/1000)
            cata   = client.get_events(starttime=UTCtime, endtime=UTCtime+60, minmagnitude=6)
            mag    = cata[0].magnitudes[0].mag
##======================== Into C-shell to run  
            EVENT = f'{yyyymmdd}{hhmmss}'

            if deg <=140:
                if not os.path.isdir(EVENT):
                    os.mkdir(EVENT)
                phase = 'SKS'
                cmd = '''
                echo %(EVENT)s %(strike)i %(dip)i %(rake)i %(phase)s %(evtlat)f %(evtlon)f %(evtdep)f %(mag)f %(deg)f %(baz)f %(stalat)f %(stalon)f > %(EVENT)s/%(phase)s_info.txt
                ''' %locals()
                os.system(cmd)
            
                cmd = '''
                csh c14.radpat_pierce_1 %(EVENT)s/%(phase)s_info.txt
                '''%locals()
                os.system(cmd)
            